## This jupyter notebook is aim to clean the leavetime tables and merge with the distance table, then build some new features:
1.pre_stop : the previous stopid 
2.diff_prog: After reordering the progrnumber on each trip. diff each row. the possible value will be
    1.0


In [141]:
import pandas as pd
import numpy as np
import re
import geopy
from geopy.distance import vincenty
import matplotlib.pyplot as plt

In [142]:
# Change txt file for specific route for 2016
# Some issue occurs in the procedure where reads txt file, which demands give the specific column name.
leavetimes_16 = pd.read_csv("leavetimes_2016_145.txt", sep=';',
                  names = ["dayofservice", "tripID", "progrnumber", "stoppointid","plannedtime_arr", "plannedtime_dep","actualtime_arr","actualtime_dep","suppressed","justificationid","note"])
leavetimes_16.head()

,dayofservice,tripID,progrnumber,stoppointid,plannedtime_arr,plannedtime_dep,actualtime_arr,actualtime_dep,suppressed,justificationid,note
0,31-MAR-16 00:00:00,2896014,66,4175,65040,65040,64992,65004,NaN,NaN,NaN
1,31-MAR-16 00:00:00,2896015,53,773,69003,69003,68981,69009,NaN,NaN,NaN
2,31-MAR-16 00:00:00,2896015,58,906,69300,69300,69244,69273,NaN,NaN,NaN
3,31-MAR-16 00:00:00,2896015,63,792,69732,69732,69749,69796,NaN,NaN,NaN
4,29-FEB-16 00:00:00,2890938,13,847,42858,42858,42928,42956,NaN,NaN,NaN


In [143]:
# Change txt file for specific route for 2017
leavetimes_17 = pd.read_csv("leavetimes_2017_145.txt", sep=';',
                  names = ["dayofservice", "tripID", "progrnumber", "stoppointid","plannedtime_arr", "plannedtime_dep","actualtime_arr","actualtime_dep","suppressed","justificationid","note"])

In [144]:
leavetimes_17.head()

,dayofservice,tripID,progrnumber,stoppointid,plannedtime_arr,plannedtime_dep,actualtime_arr,actualtime_dep,suppressed,justificationid,note
0,27-MAR-17 00:00:00,4529324,1,4320,65400,65400,65330,65330,NaN,NaN,NaN
1,27-MAR-17 00:00:00,4529324,5,1479,65900,65900,65808,65836,NaN,NaN,NaN
2,27-MAR-17 00:00:00,4529324,10,842,66907,66907,66585,66607,NaN,NaN,NaN
3,27-MAR-17 00:00:00,4529324,15,2795,67326,67326,66929,66980,NaN,NaN,NaN
4,27-MAR-17 00:00:00,4519771,56,4129,40333,40333,39989,39989,NaN,NaN,NaN


In [145]:
# concanate the table of year 16 and 17
leavetimes = pd.concat([leavetimes_16,leavetimes_17])

In [146]:
leavetimes.head()

,dayofservice,tripID,progrnumber,stoppointid,plannedtime_arr,plannedtime_dep,actualtime_arr,actualtime_dep,suppressed,justificationid,note
0,31-MAR-16 00:00:00,2896014,66,4175,65040,65040,64992,65004,NaN,NaN,NaN
1,31-MAR-16 00:00:00,2896015,53,773,69003,69003,68981,69009,NaN,NaN,NaN
2,31-MAR-16 00:00:00,2896015,58,906,69300,69300,69244,69273,NaN,NaN,NaN
3,31-MAR-16 00:00:00,2896015,63,792,69732,69732,69749,69796,NaN,NaN,NaN
4,29-FEB-16 00:00:00,2890938,13,847,42858,42858,42928,42956,NaN,NaN,NaN


In [147]:
leavetimes.dtypes

dayofservice        object
tripID               int64
progrnumber          int64
stoppointid          int64
plannedtime_arr      int64
plannedtime_dep      int64
actualtime_arr       int64
actualtime_dep       int64
suppressed         float64
justificationid    float64
note               float64
dtype: object

In [148]:
leavetimes.notnull().mean()

dayofservice       1.000000
tripID             1.000000
progrnumber        1.000000
stoppointid        1.000000
plannedtime_arr    1.000000
plannedtime_dep    1.000000
actualtime_arr     1.000000
actualtime_dep     1.000000
suppressed         0.018597
justificationid    0.018597
note               0.000000
dtype: float64

In [149]:
leavetimes.rename(columns={"plannedtime_arr":"plannedtime_arr_stop","plannedtime_dep":"plannedtime_dep_stop","actualtime_arr":"actualtime_arr_stop","actualtime_dep":"actualtime_dep_stop","suppressed":"suppressed_stop","justificationid":"justificationid_stop"},inplace=True)

In [150]:
# drop all the rows which marked abnormal behaivour
leavetimes = leavetimes[~pd.notnull(leavetimes['suppressed_stop'])]

In [151]:
leavetimes.notnull().mean()

dayofservice            1.0
tripID                  1.0
progrnumber             1.0
stoppointid             1.0
plannedtime_arr_stop    1.0
plannedtime_dep_stop    1.0
actualtime_arr_stop     1.0
actualtime_dep_stop     1.0
suppressed_stop         0.0
justificationid_stop    0.0
note                    0.0
dtype: float64

In [152]:
# All null, So drop the note column
leavetimes = leavetimes.drop(['note','suppressed_stop','justificationid_stop'], axis=1)

In [153]:
leavetimes.head()

,dayofservice,tripID,progrnumber,stoppointid,plannedtime_arr_stop,plannedtime_dep_stop,actualtime_arr_stop,actualtime_dep_stop
0,31-MAR-16 00:00:00,2896014,66,4175,65040,65040,64992,65004
1,31-MAR-16 00:00:00,2896015,53,773,69003,69003,68981,69009
2,31-MAR-16 00:00:00,2896015,58,906,69300,69300,69244,69273
3,31-MAR-16 00:00:00,2896015,63,792,69732,69732,69749,69796
4,29-FEB-16 00:00:00,2890938,13,847,42858,42858,42928,42956


In [154]:
leavetimes["new_prog"]=leavetimes["progrnumber"]
leavetimes = leavetimes.sort_values(['progrnumber'],ascending=True)
leavetimes.head()

,dayofservice,tripID,progrnumber,stoppointid,plannedtime_arr_stop,plannedtime_dep_stop,actualtime_arr_stop,actualtime_dep_stop,new_prog
751305,26-APR-17 00:00:00,4608723,1,4320,51600,51600,51550,51550,1
206590,03-MAY-16 00:00:00,3024892,1,7574,54000,54000,54130,54130,1
206592,03-MAY-16 00:00:00,3028468,1,7574,55800,55800,55790,55790,1
206593,03-MAY-16 00:00:00,3020643,1,7574,55200,55200,55134,55134,1
206594,03-MAY-16 00:00:00,3034330,1,7574,69600,69600,69597,69597,1


In [155]:
leavetimes["next_stop"] = leavetimes.groupby(["tripID","dayofservice"])["stoppointid"].shift()

In [156]:
leavetimes["diff_prog"] = leavetimes.groupby(["tripID","dayofservice"])["new_prog"].transform(pd.Series.diff)

In [157]:
leavetimes["diff_planned_arr_stop"] = leavetimes.groupby(["tripID","dayofservice"])["plannedtime_arr_stop"].transform(pd.Series.diff)
leavetimes["diff_actual_arr_stop"] = leavetimes.groupby(["tripID","dayofservice"])["actualtime_arr_stop"].transform(pd.Series.diff)

In [158]:
leavetimes = leavetimes.sort_values(['plannedtime_arr_stop'],ascending=True)
leavetimes["diff_planned_next_stop_time"] = leavetimes.groupby(["stoppointid","dayofservice"])["plannedtime_arr_stop"].transform(pd.Series.diff)

In [159]:
leavetimes['diff_prog'].unique()

array([nan,  1.,  2.,  3., 17., 22., 12.,  7.,  5., 10.,  4., 11.,  6.,
       16.,  8., 27., 13., 33., 15.,  9., 31., 18., 14., 23., 30., 35.,
       19., 36., 28., 20., 53., 29., 54., 39., 32., 44., 25., 34., 60.,
       43., 49., 21., 37., 24., 51., 38., 48., 26., 56., 42., 46., 47.,
       52., 50., 59., 45., 40., 55., 41., 68., 62., 58., 57., 71., 64.,
       69., 61.])

In [160]:
leavetimes['next_stop'].isnull().sum()

47450

In [161]:
leavetimes['diff_prog'].value_counts(normalize = True).head(5)

1.0    0.987566
2.0    0.007021
5.0    0.001177
3.0    0.001099
4.0    0.000633
Name: diff_prog, dtype: float64

In [162]:
leavetimes = leavetimes[(leavetimes["diff_prog"] == 1.0) | leavetimes["diff_prog"].isnull()]
leavetimes["diff_prog"].unique()

array([nan,  1.])

In [163]:
leavetimes['diff_prog'].isnull().sum()

47450

In [164]:
leavetimes["diff_planned_arr_stop"].isnull().sum()

47450

In [165]:
leavetimes[leavetimes["progrnumber"] == 1]["stoppointid"].unique()

array([7574, 4320, 4151, 4533, 4202, 2060,  768,  760,  767,  773,  334],
      dtype=int64)

In [166]:
leavetimes[leavetimes["progrnumber"] == 1]["next_stop"].unique()

array([nan])

In [167]:
leavetimes[leavetimes["progrnumber"] == 2]["next_stop"].unique()

array([  nan, 7574., 4320., 4151., 4533., 4202., 2060.,  768.,  760.,
        767.,  773.,  334.])

In [168]:
leavetimes.sample(10)

,dayofservice,tripID,progrnumber,stoppointid,plannedtime_arr_stop,plannedtime_dep_stop,actualtime_arr_stop,actualtime_dep_stop,new_prog,next_stop,diff_prog,diff_planned_arr_stop,diff_actual_arr_stop,diff_planned_next_stop_time
309051,13-APR-17 00:00:00,4588545,30,4571,78928,78928,79179,79208,30,7353.0,1.0,101.0,112.0,1200.0
1084112,15-APR-16 00:00:00,3005609,20,7362,58060,58060,57625,57642,20,7361.0,1.0,138.0,86.0,228.0
1209976,12-FEB-16 00:00:00,2821773,20,4205,44992,44992,45119,45119,20,4204.0,1.0,12.0,9.0,600.0
45294,16-MAY-16 00:00:00,3032201,13,4154,41667,41667,41878,41912,13,4153.0,1.0,100.0,107.0,600.0
88558,19-FEB-17 00:00:00,4382320,29,7353,52284,52284,52251,52251,29,435.0,1.0,38.0,22.0,1200.0
990167,24-FEB-16 00:00:00,2862411,15,4201,70440,70440,70456,70472,15,4416.0,1.0,62.0,64.0,600.0
711440,09-MAY-16 00:00:00,3028464,38,2063,33524,33524,33673,33688,38,2062.0,1.0,53.0,42.0,600.0
404495,28-MAY-16 00:00:00,3045443,9,4182,63729,63729,63505,63505,9,4181.0,1.0,215.0,49.0,2803.0
761906,21-MAR-17 00:00:00,4503055,47,2084,78753,78753,78692,78692,47,2070.0,1.0,45.0,72.0,1200.0
550941,10-APR-17 00:00:00,4584154,5,1479,28690,28690,28569,28587,5,1478.0,1.0,92.0,62.0,600.0


In [169]:
leavetimes.dtypes

dayofservice                    object
tripID                           int64
progrnumber                      int64
stoppointid                      int64
plannedtime_arr_stop             int64
plannedtime_dep_stop             int64
actualtime_arr_stop              int64
actualtime_dep_stop              int64
new_prog                         int64
next_stop                      float64
diff_prog                      float64
diff_planned_arr_stop          float64
diff_actual_arr_stop           float64
diff_planned_next_stop_time    float64
dtype: object

In [170]:
leavetimes["pre_stop"] = leavetimes["next_stop"].astype("object")
leavetimes["now_stop"] = leavetimes["stoppointid"].astype("object")

In [171]:
leavetimes.dtypes

dayofservice                    object
tripID                           int64
progrnumber                      int64
stoppointid                      int64
plannedtime_arr_stop             int64
plannedtime_dep_stop             int64
actualtime_arr_stop              int64
actualtime_dep_stop              int64
new_prog                         int64
next_stop                      float64
diff_prog                      float64
diff_planned_arr_stop          float64
diff_actual_arr_stop           float64
diff_planned_next_stop_time    float64
pre_stop                        object
now_stop                        object
dtype: object

In [172]:
leavetimes.sample(10)

,dayofservice,tripID,progrnumber,stoppointid,plannedtime_arr_stop,plannedtime_dep_stop,actualtime_arr_stop,actualtime_dep_stop,new_prog,next_stop,diff_prog,diff_planned_arr_stop,diff_actual_arr_stop,diff_planned_next_stop_time,pre_stop,now_stop
518599,02-MAY-17 00:00:00,4601547,34,2016,72800,72800,72250,72260,34,4636.0,1.0,81.0,35.0,600.0,4636,2016
228466,17-FEB-16 00:00:00,2851603,69,5092,52669,52669,51964,51964,69,5091.0,1.0,68.0,41.0,600.0,5091,5092
397760,02-FEB-16 00:00:00,2824674,21,4206,81294,81294,81393,81401,21,4205.0,1.0,42.0,30.0,1200.0,4205,4206
880463,12-MAY-16 00:00:00,3027451,16,756,76856,76856,76709,76721,16,2795.0,1.0,48.0,64.0,600.0,2795,756
580912,09-JAN-17 00:00:00,4094638,9,4182,70159,70159,70059,70059,9,4181.0,1.0,69.0,46.0,600.0,4181,4182
99220,01-MAY-16 00:00:00,3031852,41,4727,55346,55346,54892,54902,41,2065.0,1.0,82.0,52.0,1403.0,2065,4727
584007,30-JAN-16 00:00:00,2790740,46,2070,28265,28265,28217,28217,46,2069.0,1.0,34.0,49.0,1200.0,2069,2070
257722,15-APR-17 00:00:00,4584679,30,4571,62976,62976,62296,62312,30,7353.0,1.0,124.0,76.0,900.0,7353,4571
1163623,05-JAN-17 00:00:00,4094600,57,777,49216,49216,48753,48753,57,776.0,1.0,43.0,27.0,775.0,776,777
451760,10-FEB-17 00:00:00,4341329,58,906,50246,50246,50622,50638,58,777.0,1.0,75.0,81.0,600.0,777,906


In [173]:
leavetimes.isnull().sum()

dayofservice                       0
tripID                             0
progrnumber                        0
stoppointid                        0
plannedtime_arr_stop               0
plannedtime_dep_stop               0
actualtime_arr_stop                0
actualtime_dep_stop                0
new_prog                           0
next_stop                      47450
diff_prog                      47450
diff_planned_arr_stop          47450
diff_actual_arr_stop           47450
diff_planned_next_stop_time    37037
pre_stop                       47450
now_stop                           0
dtype: int64

In [174]:
leavetimes = leavetimes.drop(['next_stop'],axis = 1)

# the nan here mark the first stop of the trip.

In [175]:
# Change csv file for specific route
distance = pd.read_csv("distance.csv", sep=',')
distance.head()
distance.dtypes

new_stop_id      int64
next_stopID    float64
distance       float64
dtype: object

In [176]:
distance.rename(columns={"new_stop_id":"now_stop","next_stopID":"pre_stop"},inplace=True)
distance.head()

,now_stop,pre_stop,distance
0,226,NaN,NaN
1,228,226.0,261.136188
2,229,228.0,223.789101
3,227,229.0,352.070390
4,230,227.0,229.466104


In [177]:
distance["pre_stop"] = distance["pre_stop"].astype("object")
distance["now_stop"] = distance["now_stop"].astype("object")

In [178]:
final = pd.merge(leavetimes,distance, how='left', on=['pre_stop','now_stop'])

In [179]:
final.shape

(2907523, 16)

In [180]:
leavetimes.shape

(2907523, 15)

In [181]:
final["distance"].isnull().sum()

111640

In [182]:
final = final.sort_values(['progrnumber'],ascending=True)
final['cumlativesum_distance'] = final.groupby(["tripID","dayofservice"])['distance'].transform(lambda g: g.cumsum())

In [183]:
final[(final["dayofservice"] == '26-APR-17 00:00:00')& (final["tripID"]== 4608723 )]

,dayofservice,tripID,progrnumber,stoppointid,plannedtime_arr_stop,plannedtime_dep_stop,actualtime_arr_stop,actualtime_dep_stop,new_prog,diff_prog,diff_planned_arr_stop,diff_actual_arr_stop,diff_planned_next_stop_time,pre_stop,now_stop,distance,cumlativesum_distance
1338480,26-APR-17 00:00:00,4608723,1,4320,51600,51600,51550,51550,1,NaN,NaN,NaN,309.0,NaN,4320,NaN,NaN
1345405,26-APR-17 00:00:00,4608723,2,1476,51738,51738,51647,51647,2,1.0,138.0,97.0,600.0,4320,1476,617.930328,617.930328
1350029,26-APR-17 00:00:00,4608723,3,7453,51837,51837,51719,51719,3,1.0,99.0,72.0,600.0,1476,7453,426.790508,1044.720836
1354313,26-APR-17 00:00:00,4608723,4,1478,51929,51929,51746,51759,4,1.0,92.0,27.0,600.0,7453,1478,374.878837,1419.599673
1357504,26-APR-17 00:00:00,4608723,5,1479,51992,51992,51814,51814,5,1.0,63.0,68.0,600.0,1478,1479,251.838518,1671.438192
1368524,26-APR-17 00:00:00,4608723,6,7622,52222,52222,52037,52037,6,1.0,230.0,223.0,600.0,1479,7622,534.342066,2205.780258
1376363,26-APR-17 00:00:00,4608723,7,334,52391,52391,52165,52261,7,1.0,169.0,128.0,600.0,7622,334,301.202389,2506.982647
1386927,26-APR-17 00:00:00,4608723,8,406,52614,52614,52437,52518,8,1.0,223.0,272.0,600.0,334,406,668.127801,3175.110448
1392349,26-APR-17 00:00:00,4608723,9,747,52737,52737,52638,52686,9,1.0,123.0,201.0,600.0,406,747,381.018680,3556.129128
1397350,26-APR-17 00:00:00,4608723,10,842,52834,52834,52746,52768,10,1.0,97.0,108.0,600.0,747,842,356.020785,3912.149914


In [184]:
final.sample(10)

,dayofservice,tripID,progrnumber,stoppointid,plannedtime_arr_stop,plannedtime_dep_stop,actualtime_arr_stop,actualtime_dep_stop,new_prog,diff_prog,diff_planned_arr_stop,diff_actual_arr_stop,diff_planned_next_stop_time,pre_stop,now_stop,distance,cumlativesum_distance
2789487,12-JUN-16 00:00:00,3229768,53,4127,82132,82132,82676,82676,53,1.0,26.0,24.0,1200.0,4126,4127,405.794603,23375.613929
1488812,31-MAR-16 00:00:00,2896578,72,4320,54815,54815,55214,55214,72,1.0,85.0,121.0,215.0,4407,4320,509.676667,28254.723056
798702,12-MAY-17 00:00:00,4595428,10,842,40164,40164,40656,40684,10,1.0,99.0,83.0,600.0,747,842,356.020785,3912.149914
2197657,11-MAY-16 00:00:00,3028469,66,4175,67899,67899,68059,68059,66,1.0,93.0,177.0,600.0,2964,4175,425.040271,27320.778269
1327634,12-APR-17 00:00:00,4581522,52,4125,51371,51371,51908,51918,52,1.0,68.0,38.0,600.0,4124,4125,532.424039,17450.631039
2420505,16-MAY-16 00:00:00,3028470,65,7587,72335,72335,72793,72793,65,1.0,126.0,137.0,600.0,7586,7587,631.952166,25714.010444
1336196,07-JUN-16 00:00:00,3229962,40,2065,51554,51554,51578,51578,40,1.0,33.0,32.0,600.0,2064,2065,443.749730,15351.216517
2531513,16-MAR-16 00:00:00,2896582,7,4180,74780,74780,74759,74772,7,1.0,52.0,28.0,1142.0,2994,4180,321.281116,2395.316196
1588910,22-JUN-16 00:00:00,3261612,15,2795,56919,56919,56612,56612,15,1.0,47.0,46.0,600.0,848,2795,221.959984,4856.705270
2096977,07-APR-16 00:00:00,3005610,68,1443,65876,65876,65796,65821,68,1.0,179.0,369.0,300.0,325,1443,667.355100,26674.652093


In [186]:
final.to_csv("leavetimes_distance.csv",index=False)